## Calibrate an implied volatility surface - SSVI - Part 1 / static

This notebook can be cloned from [my GitHub -> research -> articles](https://github.com/se-l/research)

There are numerous papers on calibrating implied volatility surface. Here, I take a look at SSVI applied to a day of FDX just before and after their corporate earnings announcement. I want to see the quality of fit and how the model parameters behave throughout the day and an across earnings announcement. Regarding fit, I would want:  
- A model that fits actual fill IVs and is always within the bid/ask spread.
- Fitting error plotted with respect to tenor and moneyness.

SSVI stands for Surface Stochastic Volatility Inspired. The approach has been applied from: [No arbitrage global parametrization for the eSSVI volatility, A. Mingon 2022](https://arxiv.org/abs/2204.00312) which itself is extended from extended from [The extended SSVI volatility surface, Hendriks, Martini 2017](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2971502) and Gathedral (2012).

A slice of the surface's total implied variance is defined as:  
$
eSSVI(K, T) = \dfrac{1}{2} (\theta(T) + \rho(T)\psi(T)k + \sqrt{(\psi(T)k + \theta(T)\rho(T))^2 + \theta(T)^2(1-\rho(T)^2)})
$

where the log-forward-moneyness $
k = log \dfrac{K}{F_0(T)}
$, with $K$ being the option strike and $F_0$(T) the forward.

and the implied volatility can be recovered with  
$
\sigma_{imp}(K, T) = \sqrt{\dfrac{eSSVI(K,T)}{T}}
$

Let's start minimizing for those slice-wise equations without *global* surface optimization, so constraints to exclude calendar arbitrage.
 ρ2) .
e

You can skip the code blocks. Briefly, it:  
- Imports few bits so you can reproduce this
- Loads & scopes data. IVs have been pre-calculated. This is not about using QuantLib now
- Calibrates with a custom IV surface class that uses scipy.optimize for a least-squares optimizing minimizing RMSE of sample prices vs the model prices.

In [1]:
import sys, os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent))

In [2]:
import numpy as np
import pickle
import QuantLib as ql
import pandas as pd

from typing import List
from datetime import timedelta, date, time
from IPython.display import IFrame

from options.client import Client
from options.helper import get_dividend_yield, historical_volatility, get_tenor, unpack_mi_df_index
from options.typess.calibration_item import CalibrationItem, df2calibration_items
from options.typess.enums import Resolution, SecurityType, OptionRight, TickType
from options.typess.equity import Equity
from options.typess.iv_surface_essvi import CalibrationItem, IVSurface
from shared.constants import EarningsPreSessionDates, DiscountRateMarket
from shared.plotting import plot_scatter_3d
from options.typess.equity import Equity

release_date = date(2024, 6, 25)
equity = Equity('FDX')
sym = equity.symbol.lower()
moneyness_fit = 'moneyness_fwd_ln'
iv_col_nm = 'mid_price_iv'
weight_col_nm = 'volume'
min_mny = -0.3
max_mny = 0.3
rate = DiscountRateMarket
dividend_yield = get_dividend_yield(sym)
print(release_date)

2024-06-25


### Load & Scope

In [3]:
def scope_df(x: pd.DataFrame):
    return x[(x[moneyness_fit] < max_mny) & (x[moneyness_fit] > min_mny)]


def get_df_xy(df: pd.DataFrame, dates: List[date]):

    if all((c in df.columns for c in ('bid_iv', 'ask_iv'))):
        df = df[(df['bid_iv'] < 4) & (df['ask_iv'] < 4)]
    elif all((c in df.columns for c in ('fill_iv', ))):
        df = df[df['fill_iv'] < 4]

    # Scope by time of day
    v_ts = df.index.get_level_values('ts').unique()
    v_ts_scoped = v_ts[np.isin(v_ts.date, np.array(dates)) & (v_ts.hour >= 10) & (v_ts.hour <= 16)]

    return df.loc[v_ts_scoped]

In [4]:
df = pd.read_excel('df_quotes.xlsx')
df['expiry'] = df['expiry'].apply(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
df = df.set_index(['ts', 'expiry', 'strike', 'right'])
df['mid_price'] = (df['ask_close'] + df['bid_close'])/2

df_trades = pd.read_excel('df_trades.xlsx')
df_trades['expiry'] = df_trades['expiry'].apply(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
df_trades = df_trades.set_index(['ts', 'expiry', 'strike', 'right'])

In [5]:
release_date_p1 = [d.date() for d in (pd.date_range(release_date, release_date + timedelta(days=1), freq='D'))][-1]

df_q_pre = get_df_xy(df, [release_date])
df_q_post = get_df_xy(df, [release_date_p1])
df_t_pre = get_df_xy(df_trades, [release_date])
df_t_post = get_df_xy(df_trades, [release_date_p1])

### Calibrate

The RMSE is calculated based on differences in option PRICES, not IVs.
First I fitted IV directly, then prices and finally implemented the reparameterization discussed in the paper to fit an arbitrage-free surface.
The samples fitted are a union on quotes (mid price) and trades (fill price) of the entire day. Mid-prices can have fairly significant intraday deviation when spreads are large, something to be mindful when fitting snapshots.

In [6]:
%%capture
ivs_0 = IVSurface(equity)

calibration_items_bid = df2calibration_items(scope_df(df_q_pre), release_date, iv_col_nm='bid_iv', price_col_nm='bid_close', spot_col_nm='spot', rf=rate, dividend_yield=dividend_yield, weight_col_nm='vega_mid_iv')
calibration_items_ask = df2calibration_items(scope_df(df_q_pre), release_date, iv_col_nm='ask_iv', price_col_nm='ask_close', spot_col_nm='spot', rf=rate, dividend_yield=dividend_yield, weight_col_nm='vega_mid_iv')
calibration_items_quotes = df2calibration_items(scope_df(df_q_pre), release_date, iv_col_nm='mid_price_iv', price_col_nm='mid_price', spot_col_nm='spot', rf=rate, dividend_yield=dividend_yield, weight_col_nm='vega_mid_iv')
calibration_items_fill = df2calibration_items(scope_df(df_t_pre), release_date, iv_col_nm='fill_iv', price_col_nm='close', spot_col_nm='spot', rf=rate, dividend_yield=dividend_yield, weight_col_nm='vega_fill_iv', vega_col_nm='vega_fill_iv')

# To be weighted
calibration_items = calibration_items_fill + calibration_items_quotes

# Fit entire surface
for right in [OptionRight.call, OptionRight.put]:
    samples = {item.tenor_dt: item for item in calibration_items if item.right == right}
    ivs_0.calibrate_surface(right, samples)

2024-08-19 14:49:33,443 - INFO Calibrating FDX call. # tenors: 17. # samples: 17. # params: 51
2024-08-19 14:49:42,629 - INFO Calibrated in 9.19sec for FDX call total_rmse=0.7377322422072511
2024-08-19 14:49:42,631 - INFO Calibrating FDX put. # tenors: 17. # samples: 17. # params: 51
2024-08-19 14:49:47,891 - INFO Calibrated in 5.26sec for FDX put total_rmse=1.645647593939007


The only constraint: $\theta$ > 0, that is, the implied ATM variance is never negative.

##### Plot Surface
I used actual trades between 10am and 4pm. One may want to use snaps, shorter timeperiods, moving windows (getting to that in part 2)  

In [7]:
fn = 'ivs_0.html'
ivs_0.plot_surface(fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=800)

Let's look at a well matching slice and not so well matching slice. Weight is vega.

In [8]:
# Calling 
# ivs_0.plot_all_smiles(release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask)
# ivs_0.plot_all_prices(release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask)
# plots all smiles and price

In [9]:
ix_best_slice = (date(2024, 9, 20), 'call')
fn='ok_smile.html'
ivs_0.plot_smile(*ix_best_slice, release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=600)

The quotes dominate the chart. One needs to zoom in to see the layering from top to bottom of ask IV/price, traded and fitted IV/price and bid IV/price.

In [10]:
fn='ok_price.html'
ivs_0.plot_prices(*ix_best_slice, release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=600)

Now an example of a bad fit. Left side. The fitted IV is beyond the bid ask section in the low-strike region (left). A number of puts showcase this, getting better as the tenor increases. Changes to the weights, parameter bounds and simply hard conditions being within bid-ask spread may improve this. This data is fitted throughout the entire day's quotes, being within bid-ask may not yield a solution, but definitely worth exploring when fitting single market snap-shots.

In [11]:
# ix_worst_slice = ivs_t0.ps_rmse().index[ivs_t0.ps_rmse().argmax()]
ix_worst_slice = (date(2024, 6, 28), 'put')
fn='bad_smile.html'
ivs_0.plot_smile(*ix_worst_slice, release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=600)

How to the fitting parameter behave across tenors? 
- The fitting error is clearly very elevated for the shortest expiry. Note, this data is the last trading before a quarterly earnings release.
- An arbitrage free conditions is for theta to always increase. That looks good. Other conditioned will be investigated later. 

There's a sign reversal for Rho / Psi for tenor=0.36. Let's plot that slice.



In [12]:
ix_best_slice = (date(2024, 10, 18), 'call')
fn='reversed_sign_params.html'
ivs_0.plot_smile(*ix_best_slice, release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=600)

In [13]:
fn = 'ivs_0_params_rmse.html'
ivs_0.plot_params_rmse(release_date, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=800)

### Arbitrage free surface  
I implemented the reparameterization explained in the paper and set the bounds supposedly leading to an arb free surface. It's the key contribution the paper and here the surface plot calibrated on the exact same data set:

Looks okay. Reversing signs on rho and psi can lead to very similar smiles. You play with the parameters in a small app I hosted, linked from homepage.
But to analyse the behavior of the params over time, that is not really ideal and I will likely set fitting bounds for rho accordingly, in this case, smaller than zero.

In [14]:
%%capture
ivs_0_arb = IVSurface(equity)

for right in [OptionRight.call, OptionRight.put]:
    samples = {item.tenor_dt: item for item in calibration_items if item.right == right}
    ivs_0_arb.calibrate_surface_arb_free(right, samples)

2024-08-19 14:53:28,746 - INFO Calibrated in 220.04sec for FDX call total_rmse=0.8216614347268505
2024-08-19 14:53:41,206 - INFO Calibrated in 12.46sec for FDX put total_rmse=1.7898495442234124


In [15]:
fn = 'ivs_0_arb_free.html'
ivs_0_arb.plot_surface(fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=800)

Calibration takes significantly longer. My implementation is not optimized. For the purpose of studying the parameters over time, I stick with unconstrained fitting technique. Plotting the resulting params removed the earlier $\rho$ $\psi$ sign reversal and $\psi$(tenor) is overall smoother. RMSE increase by approximately 10%

In [16]:
fn = 'ivs_arb_free_0_params_rmse.html'
ivs_0_arb.plot_params_rmse(release_date, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=800)

#### Part 2: Dynamic
1. How do the fitting params behave intraday and across earnings releases.
1. Can they be used for forecasting or decent confidence intervals? 